In [1]:
###use this notebook to create databases

In [1]:
from os import system as bash
from pyspark.sql import SparkSession
from splicemachine.spark import PySpliceContext
import pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
splice = PySpliceContext(spark)

In [3]:
days = {'jan':31, 'feb':28, 'march':31, 'april':30, 'may':31, 'june': 30, 'july':31, 'aug':31, 'sep': 30, 'oct': 31, 'nov': 30, 'dec': 31}

In [4]:
def download_file(file):
    bash(f'wget https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2019/{file}.zip')
    bash(f'unzip {file}.zip')
    bash(f'hdfs dfs -copyFromLocal {file}.csv /tmp/file.csv') # Copy to HDFS
    #bash(f'rm {file}.zip') # Remove local copy
    #bash(f'rm {file}.csv') # Remove local copy
    

In [15]:
%%sql 
drop table if exists janfebmarch; 
create table janfebmarch (mmsi int, basedatetime timestamp, lat float, lon float, sog float, cog float, heading float, vesselname varchar(100), imo varchar(100), callsign varchar(100), vesseltype float, status float, length float, width float, draft float, cargo float, transcieverclass varchar(100))

Sql started successfully



In [10]:
from pyspark.sql.types import *
schema = StructType([StructField('mmsi', StringType(), True),
                     StructField('basedatetime', StringType(), True),
                     StructField('lat', StringType(), True),
                     StructField('lon', StringType(), True),
                    StructField('sog', StringType(), True),
                    StructField('cog', StringType(), True),
                    StructField('heading', StringType(), True),
                    StructField('vesselname', StringType(), True),
                    StructField('imo', StringType(), True),
                    StructField('callsign', StringType(), True),
                    StructField('vesseltype', StringType(), True),
                    StructField('status', StringType(), True),
                    StructField('length', StringType(), True),
                    StructField('width', StringType(), True),
                    StructField('draft', StringType(), True),
                    StructField('cargo', StringType(), True),
                    StructField('transcieverclass', StringType(), True)])

In [17]:
schema = splice.getSchema('janfebmarch')

In [18]:
spark.read.option("timestampFormat", "yyyy-MM-dd HH:mm:ss").csv(f'/tmp/file.csv', header=True, schema=schema).show(1)

+---------+-------------------+--------+----------+---+------+-------+----------+----+--------+----------+------+------+-----+-----+-----+----------------+
|     MMSI|       BASEDATETIME|     LAT|       LON|SOG|   COG|HEADING|VESSELNAME| IMO|CALLSIGN|VESSELTYPE|STATUS|LENGTH|WIDTH|DRAFT|CARGO|TRANSCIEVERCLASS|
+---------+-------------------+--------+----------+---+------+-------+----------+----+--------+----------+------+------+-----+-----+-----+----------------+
|338208626|2019-01-01 00:00:05|37.86505|-122.31405|0.0|-165.0|  511.0|      null|null|    null|      null|   0.0|  null| null| null| null|               B|
+---------+-------------------+--------+----------+---+------+-------+----------+----+--------+----------+------+------+-----+-----+-----+----------------+
only showing top 1 row



In [24]:
SCHEMA = 'AGUPTA'
TABLE = 'janfebmarch'

In [20]:
ha.dtypes

MMSI                  int64
BaseDateTime         object
LAT                 float64
LON                 float64
SOG                 float64
COG                 float64
Heading             float64
VesselName           object
IMO                  object
CallSign             object
VesselType          float64
Status              float64
Length              float64
Width               float64
Draft               float64
Cargo               float64
TranscieverClass     object
dtype: object

In [6]:
ha = pd.read_csv('AIS_2019_01_01.csv')

In [8]:
spark.createDataFrame(ha)

TypeError: field VesselName: Can not merge type <class 'pyspark.sql.types.DoubleType'> and <class 'pyspark.sql.types.StringType'>

In [21]:
ha.head(2)

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
0,338208626,2019-01-01T00:00:05,37.86505,-122.31405,0.0,-165.0,511.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,B
1,367420490,2019-01-01T00:00:04,31.26095,-91.65160,4.5,15.0,31.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,B


In [5]:
for i, month in enumerate(['jan', 'feb', 'march']):
    for day in range(days[month]):
        file = 'AIS_2019_' + f'{i+1:02}'  + '_' f'{day + 1:02}'
        print('Downloading file:', file)
        download_file(file)
        df = spark.read.csv(f'/tmp/file.csv', header=True)
        splice.insert(df, f'{SCHEMA}.{TABLE}', to_upper=True)
        break

IllegalArgumentException: 'Illegal pattern component: XXX'

In [11]:
df = spark.read.csv(f'/tmp/file.csv', header=True)

IllegalArgumentException: 'Illegal pattern component: XXX'

In [26]:
df = spark.read.csv(f'/tmp/AIS_2019_01_31.csv', header=True, inferSchema = True)


IllegalArgumentException: 'Illegal pattern component: XXX'

In [8]:
file = 'AIS_2019_01_03'
download_file(file)

In [9]:
df = spark.read.csv(f'/tmp/{file}.csv', header=True)

IllegalArgumentException: 'Illegal pattern component: XXX'

In [7]:
df.head(5)

[Row(MMSI='367543570', BaseDateTime='2019-01-03T00:00:00', LAT='29.14093', LON='-90.21359', SOG='0.0', COG='112.0', Heading='511.0', VesselName='FOURCHON RUNNER', IMO='IMO8973538', CallSign='WDG5125', VesselType='60', Status='0', Length='31', Width='7', Draft=None, Cargo='60', TranscieverClass='B'),
 Row(MMSI='366996350', BaseDateTime='2019-01-03T00:00:00', LAT='29.74798', LON='-95.17293', SOG='0.0', COG='32.1', Heading='511.0', VesselName='DIXIE VALOUR', IMO=None, CallSign='WDC2681', VesselType='31', Status='0', Length='21', Width='7', Draft=None, Cargo='57', TranscieverClass='B'),
 Row(MMSI='367306860', BaseDateTime='2019-01-03T00:00:00', LAT='30.38844', LON='-81.54210', SOG='2.5', COG='-152.5', Heading='511.0', VesselName='SEWELLS POINT', IMO='IMO7517856', CallSign='WYJ6341', VesselType='31', Status='15', Length='30', Width='8', Draft='4.0', Cargo=None, TranscieverClass='B'),
 Row(MMSI='367511270', BaseDateTime='2019-01-03T00:00:00', LAT='29.83960', LON='-91.83973', SOG='0.0', COG='

In [ ]:
df.withColumn('Lat_float', df['LAT'].cast('float')).show()

In [23]:
help(df)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(builtins.object)
 |  DataFrame(jdf, sql_ctx)
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`::
 |  
 |      people = spark.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the data frame, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SparkSession
 |      people = spark.read.parquet("...")
 |      department = spark.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId == department.id) \
 |        .groupBy(department.name, "gender").agg({"salary": "avg", "age

In [ ]:
pdf = df.toPandas()

In [49]:
splice.insert(df, f'{SCHEMA}.{TABLE}', to_upper=True)

In [ ]:
if not splice.tableExists(SCHEMA,TABLE):
    splice.createTable(df, f'{SCHEMA}.{TABLE}')
splice.upsert(df, f'{SCHEMA}.{TABLE}')

In [5]:
SCHEMA = 'AGUPTA'
TABLE = 'test'
#splice.createTable(df, f'{SCHEMA}.{TABLE}')
if not splice.tableExists(SCHEMA,TABLE):
    splice.createTable(df, f'{SCHEMA}.{TABLE}')
splice.upsert(df, f'{SCHEMA}.{TABLE}')

Py4JJavaError: An error occurred while calling o41.upsert.
: java.sql.SQLSyntaxErrorException: Column 'BASEDATETIME' is either not in any table in the FROM list or appears within a join specification and is outside the scope of the join specification or appears in a HAVING clause and is not in the GROUP BY list. If this is a CREATE or ALTER TABLE  statement then 'BASEDATETIME' is not a column in the target table.
	at com.splicemachine.db.impl.jdbc.SQLExceptionFactory40.getSQLException(SQLExceptionFactory40.java:102)
	at com.splicemachine.db.impl.jdbc.Util.generateCsSQLException(Util.java:289)
	at com.splicemachine.db.impl.jdbc.TransactionResourceImpl.wrapInSQLException(TransactionResourceImpl.java:502)
	at com.splicemachine.db.impl.jdbc.TransactionResourceImpl.handleException(TransactionResourceImpl.java:431)
	at com.splicemachine.db.impl.jdbc.EmbedConnection.handleException(EmbedConnection.java:1872)
	at com.splicemachine.db.impl.jdbc.ConnectionChild.handleException(ConnectionChild.ja

In [8]:
splice.insert(df, f'{SCHEMA}.{TABLE}')

Py4JJavaError: An error occurred while calling o41.insert.
: java.sql.SQLSyntaxErrorException: Column 'BASEDATETIME' is either not in any table in the FROM list or appears within a join specification and is outside the scope of the join specification or appears in a HAVING clause and is not in the GROUP BY list. If this is a CREATE or ALTER TABLE  statement then 'BASEDATETIME' is not a column in the target table.
	at com.splicemachine.db.impl.jdbc.SQLExceptionFactory40.getSQLException(SQLExceptionFactory40.java:102)
	at com.splicemachine.db.impl.jdbc.Util.generateCsSQLException(Util.java:289)
	at com.splicemachine.db.impl.jdbc.TransactionResourceImpl.wrapInSQLException(TransactionResourceImpl.java:502)
	at com.splicemachine.db.impl.jdbc.TransactionResourceImpl.handleException(TransactionResourceImpl.java:431)
	at com.splicemachine.db.impl.jdbc.EmbedConnection.handleException(EmbedConnection.java:1872)
	at com.splicemachine.db.impl.jdbc.ConnectionChild.handleException(ConnectionChild.ja

In [22]:
%%sql 

select pasta from AGUPTA.testTbl
where MMSI='367420490'

java.sql.SQLSyntaxErrorException:  Column 'PASTA' is either not in any table in the FROM list or appears within a join specification and is outside the scope of the join specification or appears in a HAVING clause and is not in the GROUP BY list. If this is a CREATE or ALTER TABLE  statement then 'PASTA' is not a column in the target table.

In [50]:
df = splice.df("select * from AGUPTA.testTbl")

In [45]:
help(df)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(builtins.object)
 |  DataFrame(jdf, sql_ctx)
 |  
 |  A distributed collection of data grouped into named columns.
 |  
 |  A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 |  and can be created using various functions in :class:`SparkSession`::
 |  
 |      people = spark.read.parquet("...")
 |  
 |  Once created, it can be manipulated using the various domain-specific-language
 |  (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 |  
 |  To select a column from the data frame, use the apply method::
 |  
 |      ageCol = people.age
 |  
 |  A more concrete example::
 |  
 |      # To create DataFrame using SparkSession
 |      people = spark.read.parquet("...")
 |      department = spark.read.parquet("...")
 |  
 |      people.filter(people.age > 30).join(department, people.deptId == department.id) \
 |        .groupBy(department.name, "gender").agg({"salary": "avg", "age

In [46]:
df.count()

14669269

In [51]:
df.count()

21383637

In [29]:
file = 'AIS_2019_01_02'
df = spark.read.csv(f'/tmp/{file}.csv', header=True)

AnalysisException: 'Path does not exist: hdfs://hdfs/tmp/AIS_2019_01_02.csv;'

In [20]:
df.head(5)

[Row(MMSI='338208626', BaseDateTime='2019-01-01T00:00:05', LAT='37.86505', LON='-122.31405', SOG='0.0', COG='-165.0', Heading='511.0', VesselName=None, IMO=None, CallSign=None, VesselType=None, Status='0', Length=None, Width=None, Draft=None, Cargo=None, TranscieverClass='B'),
 Row(MMSI='367420490', BaseDateTime='2019-01-01T00:00:04', LAT='31.26095', LON='-91.65160', SOG='4.5', COG='15.0', Heading='31.0', VesselName=None, IMO=None, CallSign=None, VesselType=None, Status='15', Length=None, Width=None, Draft=None, Cargo=None, TranscieverClass='B'),
 Row(MMSI='338060122', BaseDateTime='2019-01-01T00:00:04', LAT='30.67109', LON='-81.46722', SOG='1.9', COG='-67.4', Heading='511.0', VesselName=None, IMO=None, CallSign=None, VesselType=None, Status='0', Length=None, Width=None, Draft=None, Cargo=None, TranscieverClass='B'),
 Row(MMSI='368018190', BaseDateTime='2019-01-01T00:00:04', LAT='30.70736', LON='-88.04332', SOG='0.0', COG='-103.6', Heading='301.0', VesselName=None, IMO=None, CallSign=N

In [67]:
df.columns

['MMSI',
 'BaseDateTime',
 'LAT',
 'LON',
 'SOG',
 'COG',
 'Heading',
 'VesselName',
 'IMO',
 'CallSign',
 'VesselType',
 'Status',
 'Length',
 'Width',
 'Draft',
 'Cargo',
 'TranscieverClass']

In [10]:
%%sql 
drop table if exists testTbl; 
create table testTbl (mmsi varchar(100), basedatetime varchar(100), lat varchar(100), lon varchar(100), sog varchar(100), cog varchar(100), heading varchar(100), vesselname varchar(100), imo varchar(100), callsign varchar(100), vesseltype varchar(100), status varchar(100), length varchar(100), width varchar(100), draft varchar(100), cargo varchar(100), transcieverclass varchar(100))

SCHEMA = 'AGUPTA'
TABLE = 'testTbl'
splice.insert(df, f'{SCHEMA}.{TABLE}')

java.sql.SQLSyntaxErrorException:  Syntax error

In [16]:
%%sql 
drop table if exists testTbl; 
create table testTbl (mmsi varchar(100), basedatetime varchar(100), lat varchar(100), lon varchar(100), sog varchar(100), cog varchar(100), heading varchar(100), vesselname varchar(100), imo varchar(100), callsign varchar(100), vesseltype varchar(100), status varchar(100), length varchar(100), width varchar(100), draft varchar(100), cargo varchar(100), transcieverclass varchar(100))

Sql started successfully



In [17]:
splice.insert(df, f'{SCHEMA}.{TABLE}', to_upper=True)

In [15]:
SCHEMA = 'AGUPTA'
TABLE = 'testTbl'
splice.insert(df, f'{SCHEMA}.{TABLE}')

Py4JJavaError: An error occurred while calling o41.insert.
: java.sql.SQLSyntaxErrorException: Column 'BASEDATETIME' is either not in any table in the FROM list or appears within a join specification and is outside the scope of the join specification or appears in a HAVING clause and is not in the GROUP BY list. If this is a CREATE or ALTER TABLE  statement then 'BASEDATETIME' is not a column in the target table.
	at com.splicemachine.db.impl.jdbc.SQLExceptionFactory40.getSQLException(SQLExceptionFactory40.java:102)
	at com.splicemachine.db.impl.jdbc.Util.generateCsSQLException(Util.java:289)
	at com.splicemachine.db.impl.jdbc.TransactionResourceImpl.wrapInSQLException(TransactionResourceImpl.java:502)
	at com.splicemachine.db.impl.jdbc.TransactionResourceImpl.handleException(TransactionResourceImpl.java:431)
	at com.splicemachine.db.impl.jdbc.EmbedConnection.handleException(EmbedConnection.java:1872)
	at com.splicemachine.db.impl.jdbc.ConnectionChild.handleException(ConnectionChild.ja

In [ ]:
6714368